In [2]:
%%html

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [89]:
import pandas as pd
import numpy as np
from numpy import nan as Nan
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import re
import ipywidgets
from IPython.display import display
init_notebook_mode(connected=True)

In [90]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Analysing Diploma Exam Data - results by school

 <img src="images/Alberta_education.jpg" width="200px" align="right"/>
 Provincial diploma exam results are located here: https://education.alberta.ca/diploma-exam-administration/diploma-results/?searchMode=3 .

We will download and examin provincial results by shool xlsx spreadsheet: diploma-multiyear-sch-list-annual.xlsx

In [113]:
school_results = pd.read_excel('https://education.alberta.ca/media/3680580/diploma-multiyear-sch-list-annual.xlsx')
school_results.head()

,Diploma Course,Authority Type,Authority Code,Authority Name,School Code,School Name,2013 Sch Students Writing,2013 Sch School Mark % Exc,2013 Sch School Mark % Acc,2013 Sch School Average %,...,2017 Sch School Mark % Exc,2017 Sch School Mark % Acc,2017 Sch School Average %,2017 Sch School Standard Deviation %,2017 Sch Exam Mark % Exc,2017 Sch Exam Mark Exc Sig,2017 Sch Exam Mark % Acc,2017 Sch Exam Mark Acc Sig,2017 Sch Exam Average %,2017 Sch Exam Standard Deviation %
0,Biology 30,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,100,59.0,96.0,79.5,...,58.8,99.1,79.6,12.8,44.7,=,86.8,=,73.9,16.4
1,Chemistry 30,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,56,26.8,83.9,67.7,...,53.5,98.0,78.0,13.0,55.6,=,93.9,=,77.6,16.7
2,English Lang Arts 30-1,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,111,42.3,99.1,74.6,...,36.1,100.0,74.5,11.0,21.3,=,96.7,=,69.3,11.2
3,English Lang Arts 30-2,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,21,9.5,95.2,69.5,...,18.2,100.0,71.9,7.4,36.4,=,100.0,=,75.8,8.7
4,Mathematics 30-1,Charter,9.0,Foundations for the Future Charter Academy Cha...,12.0,FFCA High School Campus,84,36.9,95.2,73.0,...,50.9,94.7,78.0,15.2,50.9,n/a,87.7,n/a,74.5,20.2


This data set needs to be reshaped - to split year and all the metrics in separate columns. Let's reshape it and remove rows  having  "n/a" in following columns: 'Sch Students Writing','Sch School Mark % Acc','Sch School Mark % Exc','Sch Exam Mark % Exc','Sch Exam Mark % Acc'.
Examine the result:

In [106]:
school_results_reshaped  = school_results.copy()

def splitter(string):
    r = re.compile(r'\d{4}|\S.*$')
    return r.findall(string)

cols = list(school_results_reshaped)[0:6]
years = ['2013', '2014', '2015', '2016', '2017']

for year in years:
    for names in cols:
        school_results_reshaped[str(year +" "+names)] = school_results_reshaped[names]

school_results_reshaped.columns = pd.MultiIndex.from_tuples([tuple(splitter(c)) for c in school_results_reshaped.columns])
school_results_reshaped = school_results_reshaped.stack(0).reset_index(1)

school_results_reshaped.rename(columns={'level_1': "Year"}, inplace=True)
school_results_reshaped = school_results_reshaped.reset_index(drop=True)
school_results_reshaped[['Sch Students Writing','Sch School Mark % Acc','Sch School Mark % Exc','Sch Exam Mark % Exc','Sch Exam Mark % Acc']]=school_results_reshaped[['Sch Students Writing','Sch School Mark % Acc','Sch School Mark % Exc','Sch Exam Mark % Exc','Sch Exam Mark % Acc']].apply(pd.to_numeric, errors='coerce')
school_results_reshaped=school_results_reshaped.dropna(subset=['Sch Students Writing','Sch School Mark % Acc','Sch Exam Mark % Exc','Sch School Mark % Exc','Sch Exam Mark % Acc'])
school_results_reshaped=school_results_reshaped[:-5]
school_results_reshaped.head()

,Year,Authority Code,Authority Name,Authority Type,Diploma Course,Sch Exam Average %,Sch Exam Mark % Acc,Sch Exam Mark % Exc,Sch Exam Mark Acc Sig,Sch Exam Mark Exc Sig,Sch Exam Standard Deviation %,Sch School Average %,Sch School Mark % Acc,Sch School Mark % Exc,Sch School Standard Deviation %,Sch Students Writing,School Code,School Name
0,2013,9.0,Foundations for the Future Charter Academy Cha...,Charter,Biology 30,74.3,92.0,42.0,=,=,14.4,79.5,96.0,59.0,13.5,100.0,12.0,FFCA High School Campus
1,2014,9.0,Foundations for the Future Charter Academy Cha...,Charter,Biology 30,73.9,88.1,46.4,=,=,16.4,77.9,97.6,56.0,14.2,84.0,12.0,FFCA High School Campus
2,2015,9.0,Foundations for the Future Charter Academy Cha...,Charter,Biology 30,72.1,92.2,39.2,=,=,15.7,74.2,98.0,45.1,13.7,102.0,12.0,FFCA High School Campus
3,2016,9.0,Foundations for the Future Charter Academy Cha...,Charter,Biology 30,76.0,95.2,48.1,=,=,14.5,77.6,99.0,49.0,13.0,104.0,12.0,FFCA High School Campus
4,2017,9.0,Foundations for the Future Charter Academy Cha...,Charter,Biology 30,73.9,86.8,44.7,=,=,16.4,79.6,99.1,58.8,12.8,114.0,12.0,FFCA High School Campus


Let's create interactive plot - to check statistics by school for all available years.
We well use two dropdown menus - to choose school(schols are sorted alphabetically) and Diploma Course. After school is chosen - the second dropdown menu is updtated with the list of courses available for this school. We use plot.ly ( 'bar') for this visualization, you can click on any legend item and bars for this metrics will be hidden.


In [103]:
def view_schools(subject):

    result = school_results_reshaped[school_results_reshaped['School Name'] == schoolW.value]
    result = result[result['Diploma Course'] == subject]

    trace1 = go.Bar( x= result['Year'], y=result['Sch Students Writing'],
                name='Number of students taken the course', marker=dict(color='#59606D'))

    trace2 = go.Bar(x=result['Year'], y=result['Sch Students Writing']/100*result['Sch Exam Mark % Acc'],
                name='Number of students achieved acceptable standard', marker=dict(color='#ffcdd2'))

    trace3 = go.Bar(x=result['Year'], y=result['Sch Students Writing']/100*result['Sch Exam Mark % Exc'],
        name='Number of students achieved standard of excellence',marker=dict(color='#A2D5F2'))


    data = [trace1, trace2, trace3]
    layout = go.Layout(title=subject + "/" + schoolW.value,
                xaxis=dict(title='Year'),
                yaxis=dict(title='Number of students'))
    fig = go.Figure(data=data, layout=layout)

    iplot(fig)


def select_school(school):
    subjectW.options = school_results_reshaped[school_results_reshaped['School Name'] == school]['Diploma Course'].unique()
   

schoolW = widgets.Select(options=np.sort(school_results_reshaped['School Name'].unique()))
subjectW = widgets.Select(options=school_results_reshaped[school_results_reshaped['School Name'] == schoolW]['Diploma Course'].unique())

i = widgets.interactive(select_school, school=schoolW)
j = widgets.interactive(view_schools, subject=subjectW)

display(i)
display(j)

A Jupyter Widget

A Jupyter Widget

### Comparing multiple schools.

To compare schools to provincial average - lets download diploma-multiyear-province-annual.xlsx dataset with provincial results. 

In [110]:
provincial_results = pd.read_excel('https://education.alberta.ca/media/3680581/diploma-multiyear-province-annual.xlsx')
provincial_results.head()

,Diploma Course,2013 Prov Students Writing,2013 Prov School Mark % Exc,2013 Prov School Mark % Acc,2013 Prov School Average %,2013 Prov School Standard Deviation %,2013 Prov Exam Mark % Exc,2013 Prov Exam Mark Exc Sig,2013 Prov Exam Mark % Acc,2013 Prov Exam Mark Acc Sig,...,2017 Prov School Mark % Acc,2017 Prov School Average %,2017 Prov School Standard Deviation %,2017 Prov Exam Mark % Exc,2017 Prov Exam Mark Exc Sig,2017 Prov Exam Mark % Acc,2017 Prov Exam Mark Acc Sig,2017 Prov Exam Average %,2017 Prov Exam Standard Deviation %,Unnamed: 56
0,Biology 30,22429.0,42.9,96.0,74.4,13.9,32.2,+,84.4,+,...,97.0,76.3,13.7,32.3,=,84.2,-,68.7,16.9,NaN
1,Chemistry 30,16159.0,41.9,95.4,74.0,13.9,31.8,+,78.8,+,...,97.5,77.3,13.5,38.6,+,83.1,+,70.1,18.9,NaN
2,English Lang Arts 30-1,29034.0,30.7,97.1,72.1,11.8,10.4,=,85.9,=,...,97.9,73.0,11.6,11.7,=,86.5,=,64.0,12.8,NaN
3,English Lang Arts 30-2,15383.0,11.8,93.8,65.4,11.7,10.9,+,89.4,=,...,95.6,66.8,11.5,11.4,-,89.5,=,65.4,12.2,NaN
4,Français 30-1,154.0,44.8,100.0,76.5,10.1,18.2,n/a,96.8,n/a,...,97.4,75.9,10.9,18.6,n/a,98.1,n/a,70.9,10.2,NaN


This dataset needs to be reshaped as well to separate year and metrics to different columns.
Additionally  4 columns need to be  renamed to match statistics by school (from {'Prov Exam Mark % Exc', 'Prov School Mark % Exc','Prov School Mark % Acc''Prov Exam Mark % Exc','Prov Exam Mark % Acc'} to {'Sch School Mark % Exc','Sch School Mark % Acc', 'Sch Exam Mark % Exc', 'Sch Exam Mark % Acc'}).
Reshaping, renaming columns and checking the result:

In [111]:
provincial_results = pd.read_excel('https://education.alberta.ca/media/3680581/diploma-multiyear-province-annual.xlsx')  
years = []
stats = []
for value in provincial_results.columns.values[1:-1]:
    year = value[0:4]
    stat = value[5:]
    int(year)
    if year not in years:
        years.append(year)
    if stat not in stats:
        stats.append(stat)
provincial_results_reshaped = pd.DataFrame(columns=(['Diploma Course','Year'] + stats))
for ind,row in provincial_results.drop(provincial_results.index[len(provincial_results)-1]).iterrows():
    new_row = pd.DataFrame(columns=(['Diploma Course','Year'] + stats))
    new_row.loc[0] = [Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan]
    new_row.loc[0]['Diploma Course'] = row['Diploma Course']                                       
    for year in years:
        new_row.loc[0]['Year'] = year
        for stat in stats:
            new_row.loc[0][stat] = row[year+" "+stat]
        provincial_results_reshaped=provincial_results_reshaped.append(new_row)                               
provincial_results_reshaped = provincial_results_reshaped.reset_index(drop=True)

provincial_results_reshaped=provincial_results_reshaped.rename(columns = {'Prov School Mark % Exc':'Sch School Mark % Exc'})
provincial_results_reshaped=provincial_results_reshaped.rename(columns = {'Prov School Mark % Acc':'Sch School Mark % Acc'})
provincial_results_reshaped=provincial_results_reshaped.rename(columns = {'Prov Exam Mark % Exc':'Sch Exam Mark % Exc'})
provincial_results_reshaped=provincial_results_reshaped.rename(columns = {'Prov Exam Mark % Acc':'Sch Exam Mark % Acc'})
provincial_results_reshaped.head()

,Diploma Course,Year,Prov Students Writing,Sch School Mark % Exc,Sch School Mark % Acc,Prov School Average %,Prov School Standard Deviation %,Sch Exam Mark % Exc,Prov Exam Mark Exc Sig,Sch Exam Mark % Acc,Prov Exam Mark Acc Sig,Prov Exam Average %,Prov Exam Standard Deviation %
0,Biology 30,2013,22429,42.9,96,74.4,13.9,32.2,+,84.4,+,68.8,16.5
1,Biology 30,2014,21733,43.6,96.2,74.7,13.8,31.8,+,85.2,+,68.9,16.6
2,Biology 30,2015,21257,45.3,96.4,75.2,13.8,33,+,85.8,+,69.4,16.5
3,Biology 30,2016,22550,47,97.1,75.9,13.7,32.4,=,85.1,=,69.1,16.8
4,Biology 30,2017,22993,48.4,97,76.3,13.7,32.3,=,84.2,-,68.7,16.9


#### Compaing 5 schools between each other and with provincial average.

For this interactive plot  -  we create 7 dropdown menus. 5 top menus are for schools selection. The result for every school will be displayed as circles with unique colour.
Provincial average results are added as red diamonds to the plot.
2 bottom dropdown menus are created to choose Diploma Course from the list and metric. 
Available metrics are: 'Sch Exam Mark % Exc','Sch Exam Mark % Acc','Sch School Mark % Acc','Sch School Mark % Exc'. The resulting data is percentage of the students(y axix) by year (x axix).
We use plot.ly ('scatter') to diplay the data, if you click on any school in the legend - the plot will for this school will be hidden.

In [112]:

def view_schools(school1,school2,school3,school4, school5, subject, metric):

    result1 = school_results_reshaped[school_results_reshaped['School Name'] == school1]
    result1 = result1[result1['Diploma Course'] == subject]
    
    result2 = school_results_reshaped[school_results_reshaped['School Name'] == school2]
    result2 = result2[result2['Diploma Course'] == subject]
    
    result3 = school_results_reshaped[school_results_reshaped['School Name'] == school3]
    result3 = result3[result3['Diploma Course'] == subject]
    
    result4 = school_results_reshaped[school_results_reshaped['School Name'] == school4]
    result4 = result4[result4['Diploma Course'] == subject]
    
    result5 = school_results_reshaped[school_results_reshaped['School Name'] == school5]
    result5 = result5[result5['Diploma Course'] == subject]
    
    provincial_results = provincial_results_reshaped[provincial_results_reshaped['Diploma Course'] == subject]
    
    trace1 = go.Scatter(
    x = result1['Year'],
    y = result1[metric],
    mode='markers',
    opacity=0.7,
    marker={
        'size': 15,
        'line': {'width': 0.5, 'color': 'white'}
            },
    name = school1)
        
    trace2 = go.Scatter(
    x = result2['Year'],
    y = result2[metric],
    mode = 'markers',
    opacity=0.7,
    marker={
        'size': 15,
        'line': {'width': 0.5, 'color': 'white'}
            },
    name = school2)    
        
    trace3 = go.Scatter(
    x =  result3['Year'],
    y = result3[metric],
    mode = 'markers',
    opacity=0.7,
    marker={
        'size': 15,
        'line': {'width': 0.5, 'color': 'white'}
            },
    name = school3)
        
    trace4 = go.Scatter(
    x = result4['Year'],
    y = result4[metric],
    mode = 'markers',
    opacity=0.7,
    marker={
        'size': 15,
        'line': {'width': 0.5, 'color': 'white'}
            },
    name = school4)

        
    trace5 = go.Scatter(
    x = result5['Year'],
    y = result5[metric],
    mode = 'markers',
    opacity=0.7,
    marker={
        'size': 15,
        'line': {'width': 0.5, 'color': 'white'}
            },
    name = school5)
    
    trace6 = go.Scatter(
        x=provincial_results['Year'],
        y=provincial_results[metric],
        mode='markers',
        opacity=0.7,
        marker={
            'size': 15,
            'symbol':'diamond',
            'color': 'red',
            'line': {'width': 0.5, 'color': 'white'}
            },
        name='Provincial Average'
        )
        
    data = [trace1, trace2, trace3, trace4, trace5, trace6]
    layout = go.Layout(
                title=subject + " - percentage of students achived " + str(metric),
                xaxis=dict(title='Year', type='category'),
                yaxis=dict(title='% of students'))
    fig = go.Figure(data=data, layout=layout)

    iplot(fig)


interact(view_schools, school1=np.sort(school_results_reshaped['School Name'].unique()),
         school2=np.sort(school_results_reshaped['School Name'].unique()),
         school3=np.sort(school_results_reshaped['School Name'].unique()),
         school4=np.sort(school_results_reshaped['School Name'].unique()),
         school5=np.sort(school_results_reshaped['School Name'].unique()),
         subject=np.sort(school_results_reshaped['Diploma Course'].unique()),
         metric=['Sch Exam Mark % Exc','Sch Exam Mark % Acc','Sch School Mark % Acc','Sch School Mark % Exc']);

A Jupyter Widget